In [1]:
import boto3
import json
import requests
from botocore.exceptions import ClientError 

## Configure boto3

In [3]:
def configure_boto3(aws_access_key_id, aws_secret_access_key, aws_region):
    """
    Configure Boto3 with provided AWS Credentials and region
    """
    boto3.setup_default_session(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=aws_region
    )

In [9]:
aws_access_key_id = 'your-aws-access-key-here'
aws_secret_access_key = 'your-aws-secret-here'
aws_region = 'your-aws-region-here'

#Connect to AWS using Boto3
configure_boto3(aws_access_key_id, aws_secret_access_key, aws_region)

## CREATE SES CLIENT

In [10]:
ses_client = boto3.client('ses')

## CREATE SENDER IDENTITY

In [14]:
# Sender Identity is either a domain, subdomain or email that required to be verified in order to send emails.
def create_email_identity(email_address):
    try:
        response = ses_client.verify_email_identity(
            EmailAddress=email_address
        )
        
        return response
    except Exception as e:
        print(f"Create email identity error: {e}")

In [ ]:
create_email_identity("hello@domain.com")

## CREATE DOMAIN IDENTITY

In [ ]:
def create_domain_identity(domain):
    try:
        response = ses_client.verify_domain_identity(
            Domain='string'
        )
        return response
    except Exception as e:
        print(f"Create domain identity error: {e}")

In [ ]:
create_domain_identity("hello.com")

#Response syntax
# {
#     'VerificationToken': 'eoEmxw+YaYhb3h3iVJHuXMJXqeu1q1/wwmvjuEXAMPLE',
#     'ResponseMetadata': {
#         '...': '...',
#     },
# }

# VerificationToken is a TXT record that you must publish to the DNS server of your domain

## CREATE DOMAIN DKIM ATTRIBUTES

In [ ]:
# Domain Dkim Attributes are very important in order to verify domain identity
def create_domain_dkim_attributes(domain):
    try:
        response = ses_client.verify_domain_dkim(
            Domain=domain,
        )
        dkim_attributes = response['DkimAttributes']

        return dkim_attributes
    except Exception as e:
        print(f"Dkim Attributes error: {e}")

In [ ]:
create_domain_dkim_attributes("example.com")

# Response syntax
# {
#     'DkimTokens': [
#         'EXAMPLEq76owjnks3lnluwg65scbemvw',
#         'EXAMPLEi3dnsj67hstzaj673klariwx2',
#         'EXAMPLEwfbtcukvimehexktmdtaz6naj',
#     ],
#     'ResponseMetadata': {
#         '...': '...',
#     },
# }

# Using these tokens, you need to create DNS CNAME records that point to DKIM public keys that are hosted by Amazon SES.

## VERIFY DOMAIN DKIM ATTRIBUTES

In [ ]:
def verify_domain_dkim_attributes(domain):
    try:
        response = ses_client.get_identity_dkim_attributes(
            Identities=[
               domain,
            ]
        )

        return response
    except Exception as e:
        print(f"Dkim Attributes error: {e}")

In [ ]:
verify_domain_dkim_attributes(domain)

#Response
# {
#     'DkimAttributes': {
#         'string': {
#             'DkimEnabled': True|False,
#             'DkimVerificationStatus': 'Pending'|'Success'|'Failed'|'TemporaryFailure'|'NotStarted',
#             'DkimTokens': [
#                 'string',
#             ]
#         }
#     }
# }

## VERIFY EMAIL/DOMAIN IS VERIFIED

In [ ]:
## identity can be either email or domain
def check_email_domain_verified(identity):
    try:
        response = ses_client.get_identity_verification_attributes(
            Identities=[
               'identity',
            ]
        )
        verification_attributes = response['VerificationAttributes']
    except Exception as e:
        print(f"Email/Domain verification error: {e}")

In [ ]:
check_email_domain_verified("hello@domain.com")

# Response syntax
# {
#     'VerificationAttributes': {
#         'string': {
#             'VerificationStatus': 'Pending'|'Success'|'Failed'|'TemporaryFailure'|'NotStarted',
#             'VerificationToken': 'string'
#         }
#     }
# }

## DELETE EMAIL/DOMAIN IDENTITY

In [ ]:
def delete_email_domain_identity(identity_address):
    try:
        """
        Delete an email identity in Amazon SES
        """
        response = ses_client.delete_identity(
            identity=identity_address
        )
        return response
    except Exception as e:
        print(f"Delete Identity error: {e}")

In [ ]:
delete_email_domain_identity("hello@domain.com")

# Response syntax
# {
#     'ResponseMetadata': {
#         '...': '...',
#     },
# }

## SEND EMAIL

In [ ]:
def send_email(sender_email, receiver_email):
    try:
        response = client.send_email(
                        Source=sender_email,
                        Destination={
                            'ToAddresses': [
                                receiver_email,
                            ]
                        },
                        Message={
                            'Subject': {
                                'Data': 'Email Subject here',
                                'Charset': 'UTF-8'
                            },
                            'Body': {
                                'Text': {
                                    'Data': 'Text data here',
                                    'Charset': 'UTF-8'
                                },
                                'Html': {
                                    'Data': '<h1>Html data here</h1>',
                                    'Charset': 'UTF-8'
                                }
                            }
                        },
                        # ReplyToAddresses=['reply_email'],  # Set this if you want Reply in different address
                        # ConfigurationSetName='string'  # Configuration set is used to monitor email sending activity
                    )
        return response
    except Exception as e:
        print(f"Error sending email {e}")
    else:
        print("Email Sent! Message Id: ", response[MessageId])

In [ ]:
send_email("sender@email.com", "receiver@email.com")

#Response
# {
#     'MessageId': 'string'
# }

### Finally Done with Sending Email. Now lets look at Monitoring Sending activity. 

There are multiple options to track emails but the best option I found was using configuration sets and Amazon SNS 
as it monitors all email sending activity(Deliver, Reject, Complaint, etc) along with tracking email click and open. 

https://docs.aws.amazon.com/ses/latest/dg/monitor-sending-activity.html

You can either create configuration sets using Amazon Dashboard or using the following Code.

## CREATE CONFIGURATION SETS

In [ ]:
def create_configuration_sets(name):
    try:
        response = ses_client.create_configuration_set(
                        ConfigurationSet={
                            'Name': name
                        }
                    )
        return response
    except Exception as e:
        print(f"Create Configuration Error: {e}")

In [ ]:
create_configuration_set("my-first-configuration")

#Response - empty dict if sucess {}

## CREATE CONFIGURATION SET DESTINATION

In [ ]:
def create_configuration_sets_destination(configuration_name):
    try:
        response = ses_client.create_configuration_set_event_destination(
                    ConfigurationSetName=configuration_name,
                    EventDestination={
                        'Name': 'sns',
                        'Enabled': True|False,
                        'MatchingEventTypes': [
                            'send'|'reject'|'bounce'|'complaint'|'delivery'|'open'|'click'|'renderingFailure',
                        ],
                        'SNSDestination': {
                            'TopicARN': 'sns-topic-name'
                        }
                    }
                )
        return response
    except Exception as e:
        print("Create Configuration error: {e}")

In [ ]:
create_configuration_sets_destination("my-first-configuration")

### And finally Done With monitoring Email Sending acitvity. Now Check my next Repo For Amazon SNS configuration